In [1]:
sourceText=['hello', 'how are you', 'goodbye']
targetText=['bonjour', 'comment ça va', 'au revoir']


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
sourceTokenizer=Tokenizer()
sourceTokenizer.fit_on_texts(sourceText)
targetTokenizer=Tokenizer()
targetTokenizer.fit_on_texts(targetText)

In [3]:
sourceSequences=sourceTokenizer.texts_to_sequences(sourceText)
targetSequences=targetTokenizer.texts_to_sequences(targetText)
print(sourceSequences)
print(targetSequences)

[[1], [2, 3, 4], [5]]
[[1], [2, 3, 4], [5, 6]]


In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sourceSequences=pad_sequences(sourceSequences,padding='post')
targetSequences=pad_sequences(targetSequences,padding='post')
print(sourceSequences)
print(targetSequences)

[[1 0 0]
 [2 3 4]
 [5 0 0]]
[[1 0 0]
 [2 3 4]
 [5 6 0]]


In [5]:
import numpy as np 
targetSequences=np.expand_dims(targetSequences,-1)
targetSequences

array([[[1],
        [0],
        [0]],

       [[2],
        [3],
        [4]],

       [[5],
        [6],
        [0]]])

In [6]:
x=sourceSequences
print(x.shape)
x=x.reshape(x.shape[0],x.shape[1],1)
x.shape

(3, 3)


(3, 3, 1)

In [7]:
rnn_shape = (x.shape[1], 1)
rnn_shape

(3, 1)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN,Dense
model=Sequential([
    SimpleRNN(16,return_sequences=True,input_shape=rnn_shape),
    Dense(len(targetTokenizer.word_index)+1,activation='softmax')
])

C:\Users\naras\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy')

In [10]:
model.fit(x, targetSequences, epochs=100)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 2.3818
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 2.3614
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 2.3412
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 2.3211
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 2.3012
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 2.2814
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 2.2617
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 2.2422
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 2.2229
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 2.2037
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 2.1848
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 2.1660
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 2.1473
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 2.1289
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 2.1106
Epoch 16/100
1/1 ━━━━

In [25]:
def transulator(text):
    sequence=sourceTokenizer.texts_to_sequences([text])
    padded_sequence=pad_sequences(sequence,maxlen=sourceSequences.shape[1],padding='post')
    prediction=model.predict(padded_sequence)
    transulated_sentence=[]
    for wordProbs in prediction[0]:
        predictedWordIndx=np.argmax(wordProbs)
        word=targetTokenizer.index_word.get(predictedWordIndx,"")
        if word:
            transulated_sentence.append(word)
    return ' '.join(transulated_sentence)
print(transulator('how are you'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
au ça ça


In [58]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
import numpy as np

# Sample data
sourceText=['hello', 'how are you', 'goodbye']
targetText=['bonjour', 'comment ça va', 'au revoir']
# help(Tokenizer())
# Tokenizers
sourceTokenizer = Tokenizer()
targetTokenizer = Tokenizer()
sourceTokenizer.fit_on_texts(sourceText)
targetTokenizer.fit_on_texts(targetText)

# Text to sequences
sourceSequences = sourceTokenizer.texts_to_sequences(sourceText)
targetSequences = targetTokenizer.texts_to_sequences(targetText)

# Padding
sourceSequences = pad_sequences(sourceSequences,padding='post')
targetSequences = pad_sequences(targetSequences,maxlen=sourceSequences.shape[1],padding='post')

# Reshape target
targetSequences = np.expand_dims(targetSequences, -1)

# Prepare input
x = sourceSequences
x = x.reshape(x.shape[0], x.shape[1])  # No need for 3D shape when using Embedding
help(sourceTokenizer)
# Model with Embedding
model = Sequential([
    Embedding(input_dim=len(sourceTokenizer.word_index) + 1, output_dim=16),
    SimpleRNN(16, return_sequences=True),
    Dense(len(targetTokenizer.word_index) + 1, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train
model.fit(x, targetSequences, epochs=100)

# Translator function
def transulator(text):
    sequence = sourceTokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=sourceSequences.shape[1], padding='post')
    prediction = model.predict(padded_sequence)
    transulated_sentence = []
    for wordProbs in prediction[0]:
        predictedWordIndx = np.argmax(wordProbs)
        word = targetTokenizer.index_word.get(predictedWordIndx, "")
        if word:
            transulated_sentence.append(word)
    return ' '.join(transulated_sentence)

# Test
print(transulator('hello'))
print(transulator('how are you'))
print(transulator('goodbye'))


Help on Tokenizer in module keras.src.legacy.preprocessing.text object:

class Tokenizer(builtins.object)
 |  Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, analyzer=None, **kwargs)
 |
 |  DEPRECATED.
 |
 |  Methods defined here:
 |
 |  __init__(self, num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, analyzer=None, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  fit_on_sequences(self, sequences)
 |
 |  fit_on_texts(self, texts)
 |
 |  get_config(self)
 |
 |  sequences_to_matrix(self, sequences, mode='binary')
 |
 |  sequences_to_texts(self, sequences)
 |
 |  sequences_to_texts_generator(self, sequences)
 |
 |  texts_to_matrix(self, texts, mode='binary')
 |
 |  texts_to_sequences(self, texts)
 |
 |  texts_to_sequences_generator(self, texts)
 |
 |  to_json(self, **kwargs)
 |
 |  -----------------